In [1]:
%%capture
!pip install path-explain
!pip install shap

In [2]:
import os
import sys
import json
import glob
import random
from random import shuffle
import time
import re
import math
import collections
from tqdm import tqdm
from config import EPOCH, EXPERIMENT
from logger import logger
import mlflow.tensorflow
from mlflow_utils import log_mlflow

import numpy as np
import pandas as pd
import cv2

import matplotlib.pyplot as plt

import dicom
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

from sklearn import model_selection as sk_model_selection
from sklearn.metrics import roc_auc_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, Input
from tensorflow.python.ops.numpy_ops import np_config
from tensorflow.keras import Input, Model 
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import *
from tensorflow import keras 
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

#import torch
#import torch.nn as nn
#import torch.nn.functional as F
#import torch.optim as optim
#from torch.optim import lr_scheduler
#from torch.utils.data import DataLoader

from path_explain import PathExplainerTF, scatter_plot, summary_plot, softplus_activation
from path_explain.utils import set_up_environment

import shap

/usr/local/lib/python3.9/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [3]:
run_params = {'epoch': EPOCH, 'experiment': EXPERIMENT}
EPOCH = run_params.get('epoch', 40)

In [4]:
# Parameters
epoch = 1
experiment = 1


In [5]:
UNZIP_DATA = False
PROCESS_DATA = True
LOAD_DATA = False
data_dir = "./Data"
mri_types = ["FLAIR", "T1w", "T2w", "T1wCE"]
excluded_images = [109, 123, 709] # Bad images

In [6]:
%%capture
if (UNZIP_DATA):
    !unzip "Data/rsna-miccai-brain-tumor-radiogenomic-classification.zip" -d "Data"

In [7]:
if (PROCESS_DATA):
    train_df = pd.read_csv(os.path.join(data_dir, "train_labels.csv" ))
    train_df = train_df[~train_df.BraTS21ID.isin(excluded_images)]
    test_df = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))
sample_submission = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))

In [8]:
#scalling the image to {0,1}
def load_image(path, size = 224):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return cv2.resize(data, (size, size))

In [9]:
def get_all_image_paths(brats21id, image_type, folder='train'): 
    if (image_type in mri_types):
        patient_path = os.path.join(data_dir, folder, str(brats21id).zfill(5))
        paths = sorted(glob.glob(os.path.join(patient_path, image_type, "*")),key=lambda x: int(x[:-4].split("-")[-1]))
        return np.array(paths[0:len(paths):1])
    return None

def get_all_images(brats21id, image_type, folder='train', size=225):
    return [load_image(path, size) for path in get_all_image_paths(brats21id, image_type, folder)]

In [10]:
def get_all_data_for_train(train_df, image_type, image_size=32):    
    X = []
    y = []
    train_ids = []

    for i in tqdm(train_df.index):
        x = train_df.loc[i]
        images = get_all_images(int(x['BraTS21ID']), image_type, 'train', image_size)
        label = x['MGMT_value']

        X += images
        y += [label] * len(images)
        train_ids += [int(x['BraTS21ID'])] * len(images)
        assert(len(X) == len(y))
    return np.array(X), np.array(y), np.array(train_ids)

In [11]:
def get_all_data_for_test(test_df, image_type, image_size=32):
    X = []
    test_ids = []
    for i in tqdm(test_df.index):
        x = test_df.loc[i]
        images = get_all_images(int(x['BraTS21ID']), image_type, 'test', image_size)
        X += images
        test_ids += [int(x['BraTS21ID'])] * len(images)

    return np.array(X), np.array(test_ids)

In [12]:
if (PROCESS_DATA):
    X, y, trainidt = get_all_data_for_train(train_df, 'T1wCE')
    X_test, testidt = get_all_data_for_test(test_df, 'T1wCE')
    X_train, X_valid, y_train, y_valid, trainidt_train, trainidt_valid = sk_model_selection.train_test_split(X, y, trainidt, test_size=0.2, random_state=42)

  0%|          | 0/582 [00:00<?, ?it/s]

  0%|          | 1/582 [00:00<04:35,  2.11it/s]

  0%|          | 2/582 [00:00<04:19,  2.24it/s]

  1%|          | 3/582 [00:01<04:41,  2.05it/s]

  1%|          | 4/582 [00:01<04:31,  2.13it/s]

...

 97%|█████████▋| 84/87 [01:32<00:01,  2.33it/s]

 98%|█████████▊| 85/87 [01:33<00:01,  1.71it/s]

 99%|█████████▉| 86/87 [01:33<00:00,  1.97it/s]

100%|██████████| 87/87 [01:34<00:00,  1.85it/s]

100%|██████████| 87/87 [01:34<00:00,  1.09s/it]

In [13]:
if (PROCESS_DATA):
    np.save("x_train_data.npy", X_train)
    np.save("x_valid_data.npy", X_valid)
    np.save("y_train_data.npy", y_train)
    np.save("y_valid_data.npy",  y_valid)
elif (LOAD_DATA): 
    X_train = np.load("x_train_data.npy")
    X_valid = np.load("x_valid_data.npy")
    y_train = np.load("y_train_data.npy")
    y_valid = np.load("y_valid_data.npy")

In [14]:
#https://www.tensorflow.org/api_docs/python/tf/expand_dims
X_train = tf.expand_dims(X_train, axis=-1)
X_valid = tf.expand_dims(X_valid, axis=-1)

2021-12-08 08:09:55.601222: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
#https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical
y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)

print(f"X_train data: Rows={X_train.shape[0]}, Columns={X_train.shape[1]}, Type={type(X_train)}")
print(f"X_valid data: Rows={X_valid.shape[0]}, Columns={X_valid.shape[1]}, Type={type(X_valid)}")
print(f"y_train data: Rows={y_train.shape[0]}, Columns={y_train.shape[1]}, Type={type(y_train)}")
print(f"y_valid data: Rows={y_valid.shape[0]}, Columns={y_valid.shape[1]}, Type={type(y_valid)}")

X_train data: Rows=41019, Columns=32, Type=<class 'tensorflow.python.framework.ops.EagerTensor'>
X_valid data: Rows=10255, Columns=32, Type=<class 'tensorflow.python.framework.ops.EagerTensor'>
y_train data: Rows=41019, Columns=2, Type=<class 'numpy.ndarray'>
y_valid data: Rows=10255, Columns=2, Type=<class 'numpy.ndarray'>


## Models

In [16]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor="auc", mode='max', patience=3)
roc_auc = tf.keras.metrics.AUC(name='roc_auc', curve='ROC')
checkpoint_filepath = 'models/ls/best_models'
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor="auc",
    mode="max",
    save_best_only=True,
    save_freq="epoch",
    verbose=1,
)

In [17]:
def eval_model(model, model_name):
    y_pred = model.predict(X_valid)
    pred = np.argmax(y_pred, axis=1)
    res = pd.DataFrame(trainidt_valid)
    res[1] = pred
    res.columns = ["BraTS21ID", "MGMT_value"]
    result = res.groupby("BraTS21ID", as_index=False).mean()
    result = result.merge(train_df, on="BraTS21ID")
    auc = roc_auc_score(result.MGMT_value_y, result.MGMT_value_x,)
    print(f"Validation AUC={auc}")
    log_mlflow(run_params=run_params, model=model1, model_name=model_name, y_true=valid, y_pred=pred)

In [18]:
def get_model1(name='model1'):
    np.random.seed(0)
    random.seed(42)
    tf.random.set_seed(42)

    inputs = keras.Input(shape=X_train.shape[1:])

    h = layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)

    h = layers.Conv2D(64, kernel_size=(4, 4), activation="relu", name="Conv_1")(h)
    h = layers.MaxPool2D(pool_size=(2, 2))(h)

    h = layers.Conv2D(32, kernel_size=(2, 2), activation="relu", name="Conv_2")(h)
    h = layers.MaxPool2D(pool_size=(1, 1))(h)

    h = layers.Dropout(0.1)(h)

    h = layers.Flatten()(h)
    h = layers.Dense(32, activation="relu")(h)

    output = keras.layers.Dense(2, activation="softmax")(h)

    model = keras.Model(inputs, output, name=name)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=[roc_auc])
    
    return model

In [19]:
keras.backend.clear_session()
model1 = get_model1()
model1.summary()
model1.fit(x=X_train, y = y_train, batch_size=64, epochs=40, 
                        callbacks=[model_checkpoint_cb, early_stopping_cb],
                        validation_data=(X_valid, y_valid))

Model: "model1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
rescaling (Rescaling)        (None, 32, 32, 1)         0         
_________________________________________________________________
Conv_1 (Conv2D)              (None, 29, 29, 64)        1088      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
Conv_2 (Conv2D)              (None, 13, 13, 32)        8224      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 13, 13, 32)        0    

2021-12-08 08:10:12.033424: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/40


  1/641 [..............................] - ETA: 26:21 - loss: 0.6931 - roc_auc: 0.5316

  3/641 [..............................] - ETA: 26s - loss: 0.6883 - roc_auc: 0.5709  

...

267/641 [===========>..................] - ETA: 32s - loss: 0.2455 - roc_auc: 0.9503

268/641 [===========>..................] - ETA: 32s - loss: 0.2453 - roc_auc: 0.9503

269/641 [===========>..................] - ETA: 32s - loss: 0.2452 - roc_auc: 0.9504

270/641 [===========>..................] - ETA: 32s - loss: 0.2452 - roc_auc: 0.9504

271/641 [===========>..................] - ETA: 32s - loss: 0.2452 - roc_auc: 0.9504

In [ ]:
eval_model(model1, "model_1")

In [ ]:
def get_model2(name='model2'):
    
    inputs_model = keras.Input(shape=X_train.shape[1:])
        
    input_dim = (32, 32, 1)
    input_tensor = Input(input_dim, name='input2d')
    efnet = DenseNet121(weights=None, include_top = False, input_shape=(32, 32, 3))
    mapping3feat = Conv2D(3, (3, 3), padding='same', use_bias=False)(input_tensor)
    
    output = efnet(mapping3feat)
    output = GlobalAveragePooling2D()(output)
    output = Dense(2, activation='sigmoid')(output)
    
    model = Model(input_tensor, output)
    roc_auc = tf.keras.metrics.AUC(name='roc_auc', curve='ROC')
    
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
        optimizer=Adam(learning_rate=1e-3),
        metrics=[roc_auc, tf.keras.metrics.BinaryAccuracy(name='bacc')],)
    
    
    return model

In [ ]:
keras.backend.clear_session()
model2 = get_model2() 
model2.summary()
checkpoint_cb = keras.callbacks.ModelCheckpoint("model2.h5", monitor='val_auc', mode='max', save_best_only=True)
model2.fit(x=X_train, y = y_train, batch_size=64, epochs=5,
           validation_data=(X_valid, y_valid), 
           callbacks=[model_checkpoint_cb, early_stopping_cb])

In [ ]:
eval_model(model2)

In [ ]:
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers

def get_model3(name="model3"):
    
    inputs_model = keras.Input(shape=X_train.shape[1:])
    input_dim = (32, 32, 1)
    input_tensor = Input(input_dim, name='input2d')
    
    # --- Define model
    conv = lambda x, filters,kernel_size, strides : layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    norm = lambda x : layers.BatchNormalization()(x)
    relu = lambda x : layers.ReLU()(x)
    pool = lambda x : layers.MaxPool2D(pool_size=(3, 3), strides=1, padding='same')(x)
    conv2 = lambda filters, x : relu(norm(conv(x, filters, kernel_size=(3,3),strides=(2, 2))))
    
    # --- Define 1x1, 3x3 and 5x5 convs
    conv1 = lambda filters, x : relu(norm(conv(x, filters, kernel_size=(1, 1),strides=1)))
    conv3 = lambda filters, x : relu(norm(conv(x, filters, kernel_size=(3, 3),strides=1)))
    conv5 = lambda filters, x : relu(norm(conv(x, filters, kernel_size=(5, 5),strides=1)))
    mpool = lambda x : relu(norm(pool(x)))
    
    # --- Define projection
    proj = lambda filters, x : layers.Conv2D(
        filters=filters, 
        strides=1, 
        kernel_size=(1, 1),
        padding='same')(x)
    
    tran = lambda x, filters, kernel_size, strides : layers.Conv2DTranspose(
        filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    
    tran_bloc = lambda filters, x : relu(norm(tran(x, filters, kernel_size=(3,3), strides=(2, 2))))
    
    # --- Define concat function
    concat = lambda a, b : layers.Concatenate()([a, b])
    
    # --- Define inception function
    def inception(filters, prev_layer):
        b1 = proj(filters, prev_layer)
        p1 = conv1(filters, prev_layer)
        p2 = conv3(filters, b1)
        p3 = conv5(filters, b1)
        p4 = proj(filters, mpool(prev_layer))
        return layers.Concatenate()([p1,p2,p3,p4])
    
    
    # --- Define contracting layers
    l1 = conv1(32, input_tensor)
    l2 = inception(16, conv2(32, l1))
    l3 = inception(32, conv2(64, l2))
    l4 = inception(64, conv2(128, l3))
    
    # --- Define expanding layers
    l5 = tran_bloc(128, l4)
    l6 = tran_bloc(64, inception(64, conv3(128,concat(l3, l5))))
    l7 = tran_bloc(32, inception(32, conv3(64, concat(l2, l6))))
    l8 = inception(4, l7)
    
    # --- Create logits
    output = layers.Conv2D(filters=2, kernel_size=(3, 3),padding='same')(l8)
    output = GlobalAveragePooling2D()(output)
    output = Dense(1, activation='sigmoid')(output)
    
    # --- Create model
    model = Model(inputs=input_tensor, outputs=output)
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-3),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
        metrics=[roc_auc, tf.keras.metrics.BinaryAccuracy(name='bacc')],)
    
    model.summary()
    
    return model

In [ ]:
keras.backend.clear_session()
model3 = get_model3()
checkpoint_cb = keras.callbacks.ModelCheckpoint("model2.h5", monitor='val_auc', mode='max', save_best_only=True)
model3.fit(x=X_train, y = y_train, batch_size=64, epochs=20,
           validation_data=(X_valid, y_valid), 
           callbacks=[model_checkpoint_cb, early_stopping_cb])

## Submission 

In [ ]:
result.columns=['BraTS21ID','MGMT_value']

result2 = result.groupby('BraTS21ID',as_index=False).mean()
result2['BraTS21ID'] = sample_submission['BraTS21ID']

# Rounding... 0.907866 -> 0.9
result2['MGMT_value'] = result2['MGMT_value'].apply(lambda x:round(x*10)/10)
# result2['MGMT_value'] = result2['MGMT_value'] # No rounding
result2.to_csv('submission_vd.csv',index=False)
result2